Audio Transcriber Colab Notebook
このノートブックは音声ファイルから文字起こしを生成するためのものです

In [ ]:
# ライブラリをインストール（セッションの再起動が必要になる場合があります）
!pip install google-cloud-secret-manager openai anthropic pydub tqdm httpx==0.26.0 openai==1.54.0 chardet
!apt-get install -y ffmpeg

In [ ]:
# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 必要なモジュールをダウンロード
!wget -O /content/audio_transcriber_parallel.py \
  https://raw.githubusercontent.com/tiaroka/whisper-claude-transcriber/main/audio_transcriber_parallel.py

!wget -O /content/colab_local_compatibility.py \
  https://raw.githubusercontent.com/tiaroka/whisper-claude-transcriber/main/colab_local_compatibility.py

In [ ]:
# colab_local_compatibility.pyの内容を作成
%%writefile /content/colab_local_compatibility.py
import os
import sys
from pathlib import Path
from google.colab import auth
from google.cloud import secretmanager

In [ ]:
def is_running_in_colab():
    """Colabで実行されているかどうかを判定する。"""
    return 'google.colab' in sys.modules

In [ ]:
def setup_environment():
    """環境に応じた設定を行い、必要なパスを返す。"""
    if is_running_in_colab():
        # Google Colab環境の場合（パスは必要に応じて調整してください）
        base_input_dir = "/content/drive/MyDrive/Transcripter/Input"
        base_output_dir = "/content/drive/MyDrive/Transcripter/Output"
        
        # フォルダが存在しない場合は作成
        os.makedirs(base_input_dir, exist_ok=True)
        os.makedirs(base_output_dir, exist_ok=True)
    else:
        # ローカル環境の場合
        script_dir = Path(__file__).parent.absolute()
        base_input_dir = os.path.join(script_dir, "input")
        base_output_dir = os.path.join(script_dir, "output")
        
        # フォルダが存在しない場合は作成
        os.makedirs(base_input_dir, exist_ok=True)
        os.makedirs(base_output_dir, exist_ok=True)
    
    return {
        "base_input_dir": base_input_dir,
        "base_output_dir": base_output_dir
    }

In [ ]:
def get_api_keys():
    """環境に応じてAPIキーを設定する。"""
    if is_running_in_colab():
        # Google Colab環境の場合、Secret Managerからキーを取得
        try:
            auth.authenticate_user()
            
            # ここでプロジェクトIDを設定
            project_id = input("Google Cloud プロジェクトIDを入力してください: ")
            
            # OpenAIのAPIキー
            openai_secret_name = input("OpenAI APIキーのSecret名を入力してください (デフォルト: OPENAI_API_KEY): ") or "OPENAI_API_KEY"
            
            # AnthropicのAPIキー
            anthropic_secret_name = input("Anthropic APIキーのSecret名を入力してください (デフォルト: ANTHROPIC_API_KEY): ") or "ANTHROPIC_API_KEY"
            
            # Secret Managerからキーを取得
            client = secretmanager.SecretManagerServiceClient()
            
            # OpenAI APIキー
            name = f"projects/{project_id}/secrets/{openai_secret_name}/versions/latest"
            response = client.access_secret_version(request={"name": name})
            openai_api_key = response.payload.data.decode("UTF-8")
            os.environ["OPENAI_API_KEY"] = openai_api_key
            print("OpenAI API key has been set.")
            
            # Anthropic APIキー
            name = f"projects/{project_id}/secrets/{anthropic_secret_name}/versions/latest"
            response = client.access_secret_version(request={"name": name})
            anthropic_api_key = response.payload.data.decode("UTF-8")
            os.environ["ANTHROPIC_API_KEY"] = anthropic_api_key
            print("Anthropic API key has been set.")
            
        except Exception as e:
            print(f"Error getting API keys from Secret Manager: {e}")
            # フォールバック: 直接入力
            get_api_keys_from_input()
    else:
        # ローカル環境の場合
        try:
            from dotenv import load_dotenv
            # .envファイルが存在すればロード
            if os.path.exists(".env"):
                load_dotenv()
                if os.getenv("OPENAI_API_KEY") and os.getenv("ANTHROPIC_API_KEY"):
                    print("API keys loaded from .env file.")
                    return
        except ImportError:
            pass
        
        # 環境変数からキーを取得
        if os.getenv("OPENAI_API_KEY") and os.getenv("ANTHROPIC_API_KEY"):
            print("API keys found in environment variables.")
            return
        
        # 上記の方法で取得できなかった場合は直接入力
        get_api_keys_from_input()

In [ ]:
def get_api_keys_from_input():
    """ユーザー入力からAPIキーを取得する。"""
    openai_api_key = input("OpenAI APIキーを入力してください: ")
    os.environ["OPENAI_API_KEY"] = openai_api_key
    
    anthropic_api_key = input("Anthropic APIキーを入力してください: ")
    os.environ["ANTHROPIC_API_KEY"] = anthropic_api_key
    
    print("API keys have been set from user input.")

In [ ]:
# APIキー設定（別の方法）
# 直接設定することもできます
def set_api_keys_manually():
    """APIキーを手動で設定する"""
    # OpenAI APIキー
    openai_api_key = "" # ここにAPIキーを入力
    if openai_api_key:
        os.environ["OPENAI_API_KEY"] = openai_api_key
        print("OpenAI API key has been set manually.")
    else:
        openai_api_key = input("OpenAI APIキーを入力してください: ")
        os.environ["OPENAI_API_KEY"] = openai_api_key
    
    # Anthropic APIキー
    anthropic_api_key = "" # ここにAPIキーを入力
    if anthropic_api_key:
        os.environ["ANTHROPIC_API_KEY"] = anthropic_api_key
        print("Anthropic API key has been set manually.")
    else:
        anthropic_api_key = input("Anthropic APIキーを入力してください: ")
        os.environ["ANTHROPIC_API_KEY"] = anthropic_api_key

In [ ]:
# プログラム実行
import audio_transcriber_parallel as transcriber
# メインプログラムを実行
transcriber.main()